<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>Basado en: &copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Modificado el 2018-1.</font>
</p>

# Tabla de contenidos
1. [Polimorfismo](#Polimorfismo)
    1. [Overloading de operadores en Python](#Overloading-de-operadores-en-Python)
    2. [`__repr__` vs `__str__`](#__repr__-vs-__str__)
2. [Duck typing](#Duck-typing)

## Polimorfismo

El polimorfismo "se refiere a la propiedad por la que es posible enviar mensajes sintácticamente iguales a objetos de tipos distintos" ([Wikipedia](https://es.wikipedia.org/wiki/Polimorfismo_(inform%C3%A1tica), 2017)). Básicamente es utilizar distintos objetos de tipos distintos con la misma *interfaz*. _Overriding_ y _overloading_ son dos maneras de hacer polimorfismo.

- **Overriding**: ocurre cuando se implementa un método en una subclase que "invalida" la implementación del mismo método en la super clase
   
- **Overloading**: es la capacidad de definir un método con el mismo nombre pero con distinto número y tipo de argumentos. Es la capacidad de una función de ejecutar distintas acciones dependiendo del tipo y número de argumentos que recibe. 
  
Python no soporta _function overloading_. No se puede definir la función más de una vez con distintos tipos y números de argumentos y esperar que ambas definiciones sean consideradas por el programa. Sin embargo, se puede "simular" usando algunos parámetros con valores por defecto o número de argumentos variables.
   

#### Ejemplo

La clase `Variable` representa un conjunto de datos cualquiera, mientras que la subclase `Ingresos` contiene un método para calcular el valor "representante" (algo así como el promedio, mediana, moda, etc.). Ocurre lo mismo con las subclases `Comuna` y `Puesto`: 

- Si los datos corresponden a ingresos, el representante es el promedio. 
- Si los datos corresponden a la comuna, el representante es la comuna que más se repite. 
- Si los datos corresponden al puesto de trabajo, entonces el representante es el que tiene el puesto más alto según la jerarquía especificada en el diccionario "categorías".

In [5]:
import statistics

class Variable:
    
    def __init__(self, data):
        self.data = data

    def representante(self):
        pass


class Ingresos(Variable):
    
    def representante(self):
        return statistics.mean(self.data)


class Comuna(Variable):
    
    def representante(self):
        return statistics.mode(self.data)


class Puesto(Variable):
    
    # Ordenadas de menor a mayor
    categorias = ('Alumno en Practica', 'Analista', 'SubGerente', 'Gerente')

    def representante(self):
        # Paso 1: Transformar la lista en lista de números, donde 0 es alumno en práctica y 3 gerente
        datos = []
        for cargo in self.data:
            datos.append(Puesto.categorias.index(cargo))
        # Paso 2: Vemos cuál es el máximo
        maximo = max(datos)
        # Paso 3: Retornar cargo asociado
        return Puesto.categorias[maximo]

In [6]:
lista_pesos = Ingresos([50, 80, 90, 150, 45, 65, 78, 89, 59, 77, 90])
lista_comunas = Comuna(['Providencia', 'Macul' , 'LaReina' ,'Santiago', 'Providencia', 'PuenteAlto',
                        'Macul', 'Santiago', 'Santiago'])
lista_puestos = Puesto(['SubGerente', 'Analista','SubGerente','Analista','Alumno en Practica',
                        'Alumno en Practica'])

print(lista_pesos.representante())
print(lista_comunas.representante())
print(lista_puestos.representante())

79.36363636363636
Santiago
SubGerente


### Overloading de operadores en Python

Existen muchos operadores en Python que funcionan para varias de las clases "built-in". Por ejemplo, el operador "+" puede sumar dos números, concatenar dos strings, mezclar dos listas, etc. dependiendo de la clase con la que estemos trabajando:

In [3]:
a = [1, 2, 3, 4]
b = [5, 6, 7, 8]
print(a + b)
c = "Hola"
d = " Mundo"
print(c + d)

[1, 2, 3, 4, 5, 6, 7, 8]
Hola Mundo


Nosotros también podemos personalizar el método `__add__` para que funcione en algún tipo de clase específica que necesitemos. Por ejemplo, supongamos una clase que representa un carro de compra:

In [4]:
from collections import namedtuple


Productos = namedtuple('Productos', ['pan', 'leche', 'agua'])

class Carro(Productos):
    '''
    Un carro de compras lo representaremos como un namedtuple
    '''
    
    def __add__(self, otro):
        # Nota: Esto se puede escribir mucho más elegante con los contenidos de programación funcional
        valores = tuple(self)
        valores_otro = tuple(otro)
        valores_sumados = []
        
        for i in range(len(self)):
            valores_sumados.append(valores[i] + valores_otro[i])
            
        return Carro(*valores_sumados)

In [5]:
carro_1 = Carro(1, 2, 3)
carro_2 = Carro(3, 4, 5)
carro_sumado = carro_1 + carro_2
print(carro_sumado)

Carro(pan=4, leche=6, agua=8)


De la misma forma, podemos personalizar la mayoría de los operadores. Por ejemplo, para personalizar el operador "menor que" implementamos `__lt__` (del inglés _less than_):

In [6]:
import math

class Vector: 
    """Vector desde el origen"""
    
    def __init__(self, x, y): 
        self.x = x 
        self.y = y
        
    @property
    def magnitud(self):
        return math.sqrt(self.x ** 2 + self.y ** 2)
    
    def __lt__(self, otro_punto):
        return self.magnitud < otro_punto.magnitud

v1 = Vector(2,4)
v2 = Vector(8,3)
print(v1 < v2)

True


### `__repr__` vs `__str__`

Podemos implementar los métodos `__repr__` y `__str__` para entregar una representación en texto de nuestro objeto. Estos métodos deben retornar un string, el que podrá ser usado por la función `print`. Si se implementan ambos, `print` utiliza `__str__`.

La diferencia entre  `__str__` y `__repr__` es sutil, puesto si bien ambos devuelven una representación del objeto en forma de _string_, cada representación persigue un objetivo distinto. Por una parte, `__str__` busca devolver una representación legible (_human-readable_) del objeto. Es como un usuario del programa quisiera leer esa información. Por otra parte, `__repr__` tiene por objetivo ofrecer una representación completa y sin ambigüedades del objeto. Es como un desarrollador del programa quisiera leer esa información.

In [7]:
class Fraccion: 

    def __init__(self, numerador, denominador): 
        self.numerador = numerador 
        self.denominador = denominador
        
    def __repr__(self):
        return f"Fraccion({self.numerador}, {self.denominador})"
    
    def __str__(self):
        return f"{self.numerador} / {self.denominador}"
    
frac = Fraccion(3, 4)

In [8]:
repr(frac)

'Fraccion(3, 4)'

In [9]:
str(frac)

'3 / 4'

In [10]:
print(frac)

3 / 4


Si no implementamos el método `__str__`, `print` va a imprimir el string que retorna la función `__repr__`

In [11]:
class Fraccion: 

    def __init__(self, numerador, denominador): 
        self.numerador = numerador 
        self.denominador = denominador
        
    def __repr__(self):
        return f"Fraccion({self.numerador}, {self.denominador})"
    
frac = Fraccion(3, 4)
print(frac)

Fraccion(3, 4)


## _Duck typing_

> "If it walks like a duck and quacks like a duck then it is a duck" 
(no importa el tipo de objeto si contiene la acción)
 
_Duck typing_ es una característica de algunos lenguajes que hace que el polimorfismo sea menos atractivo, ya que el lenguaje por sí sólo es capaz de generar comportamiento polimórfico sin la necesidad de implementar el polimorfismo a través de la herencia. 

In [12]:
class Pato:
    
    def gritar(self):
        print("Quack!")
        
    def caminar(self):
        print("Caminando como un pato")        
    
class Persona:
    
    def gritar(self):
        print("Ahhh!")
        
    def caminar(self):
        print("Caminando como un humano")        

        
def activar(pato): # Esto en otro tipo de lenguaje obligaría a que pato sea del tipo "Pato", por lo tanto
    pato.gritar() # la función activar no podría ser llamada con un argumento tipo "Persona"
    pato.caminar()

donald = Pato()
juan = Persona()
activar(donald)
activar(juan)

Quack!
Caminando como un pato
Ahhh!
Caminando como un humano
